## VacationPy

### Starter Code to Import Libraries and Load Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,angaur state,6.9060,134.1300,27.90,81,100,8.70,PW,1689535135
1,1,adamstown,-25.0660,-130.1015,20.79,79,100,3.15,PN,1689534908
2,2,port-aux-francais,-49.3500,70.2167,2.00,54,92,12.92,TF,1689534898
3,3,stanley,54.8680,-1.6985,16.77,74,20,5.66,GB,1689535137
4,4,ust-nera,64.5667,143.2000,9.96,78,12,3.86,RU,1689535137


In [5]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = 'OSM',
    size= 'Humidity',
    color='City')
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [5]:
city_data_df = city_data_df[(city_data_df['Max Temp'] < 20) & (city_data_df['Max Temp'] > 10) & (city_data_df['Wind Speed'] > 6)]
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,edinburgh of the seven seas,-37.0676,-12.3116,13.01,78,10,13.04,SH,1689535142
62,62,margaret river,-33.9500,115.0667,15.69,80,62,6.64,AU,1689535066
89,89,keflavik,64.0049,-22.5624,13.95,54,0,9.77,IS,1689535173
160,160,ceerigaabo,10.6162,47.3680,18.86,66,50,8.74,SO,1689535207
218,218,shipston on stour,52.0606,-1.6278,17.68,72,9,6.15,GB,1689535232
305,305,malmberget,67.1753,20.6549,12.31,94,75,6.17,SE,1689535276
324,324,coquimbo,-29.9533,-71.3436,14.85,82,0,7.20,CL,1689535287
330,330,dauphin,51.1494,-100.0502,16.08,72,100,6.17,CA,1689535292
411,411,tromso,69.6496,18.9570,16.01,77,75,6.69,NO,1689535331
438,438,villa gesell,-37.2639,-56.9730,10.03,69,86,8.38,AR,1689535344


In [6]:
hotel_df = city_data_df.copy()

In [7]:
hotel_df = hotel_df[['City', "Country", "Lat", "Lng", "Humidity"]]
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,edinburgh of the seven seas,SH,-37.0676,-12.3116,78,
62,margaret river,AU,-33.9500,115.0667,80,
89,keflavik,IS,64.0049,-22.5624,54,
160,ceerigaabo,SO,10.6162,47.3680,66,
218,shipston on stour,GB,52.0606,-1.6278,72,
305,malmberget,SE,67.1753,20.6549,94,
324,coquimbo,CL,-29.9533,-71.3436,82,
330,dauphin,CA,51.1494,-100.0502,72,
411,tromso,NO,69.6496,18.9570,77,
438,villa gesell,AR,-37.2639,-56.9730,69,


In [8]:
# Print a message to follow up the hotel search
print("Starting hotel search")
radius = 10000
params = {"categories":'accommodation.hotel',
          "apiKey":geoapify_key}
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
keflavik - nearest hotel: Núpan Deluxe
ceerigaabo - nearest hotel: Sanaag Hotel
shipston on stour - nearest hotel: The Old Mill
malmberget - nearest hotel: Scandic hotel
coquimbo - nearest hotel: Hotel Iberia
dauphin - nearest hotel: No hotel found
tromso - nearest hotel: Clarion Collection Hotel Aurora
villa gesell - nearest hotel: Hotel El Faro
cardigan - nearest hotel: Red Lion Inn
vadso - nearest hotel: Scandic Vadsø
newport - nearest hotel: Queens Hotel
hammerfest - nearest hotel: Thon Hotel Hammerfest
flekkefjord - nearest hotel: Grand hotell Flekkefjord


,City,Country,Lat,Lng,Humidity,Hotel Name
18,edinburgh of the seven seas,SH,-37.0676,-12.3116,78,No hotel found
62,margaret river,AU,-33.9500,115.0667,80,Margaret River Hotel
89,keflavik,IS,64.0049,-22.5624,54,Núpan Deluxe
160,ceerigaabo,SO,10.6162,47.3680,66,Sanaag Hotel
218,shipston on stour,GB,52.0606,-1.6278,72,The Old Mill
305,malmberget,SE,67.1753,20.6549,94,Scandic hotel
324,coquimbo,CL,-29.9533,-71.3436,82,Hotel Iberia
330,dauphin,CA,51.1494,-100.0502,72,No hotel found
411,tromso,NO,69.6496,18.9570,77,Clarion Collection Hotel Aurora
438,villa gesell,AR,-37.2639,-56.9730,69,Hotel El Faro


In [9]:
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = 'OSM',
    size= 'Humidity',
    color='City',
    hover_cols = ['Hotel Name', 'Country'])
map_plot_2

C:\Users\gregw\anaconda3\envs\dev\Lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)